# ML-7. Прогнозирование биологического ответа HW-3.2 

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, cross_validate, cross_val_score
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
from hyperopt import hp, fmin, tpe, Trials

import warnings
warnings.filterwarnings("ignore")


random_state = 42

In [2]:
data = pd.read_csv("data/_train_sem09.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3751 entries, 0 to 3750
Columns: 1777 entries, Activity to D1776
dtypes: float64(942), int64(835)
memory usage: 50.9 MB


In [3]:
X, y = data.drop("Activity", axis=1), data["Activity"]
kf = KFold(n_splits=10)

Для начала подберем параметры и одучим линейные модели

In [4]:


paramgrid_log = [{"penalty": ['l2'], "solver": ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'], "C":  range(1, 10)},
                 {"penalty": ['l1'], "solver": ['liblinear', 'saga'], "C":  range(1, 10)}
                 ]

# подбор с помощью RandomizedSearchCV
random = RandomizedSearchCV(
    estimator= LogisticRegression(random_state=random_state, n_jobs=-1, max_iter=50),
    param_distributions=paramgrid_log,
    cv=kf,
    n_iter=50,
    n_jobs=-1,
    random_state=random_state
)
%time random.fit(X,y)
score_random = cross_val_score(random.best_estimator_, X, y, cv=kf, scoring="f1", n_jobs=-1).mean()
print("RandomizedSearchCV: Наилучшие значения гиперпараметров {}".format(random.best_params_))

# подбор с помощью GridSearchCV
grid = GridSearchCV(
    estimator= LogisticRegression(random_state=random_state, n_jobs=-1, max_iter=50),
    param_grid=paramgrid_log,
    cv=kf,
    n_jobs=-1
)
%time grid.fit(X,y)
score_grid = cross_val_score(grid.best_estimator_, X, y, cv=kf, scoring="f1", n_jobs=-1).mean()
print("GridSearchCV: Наилучшие значения гиперпараметров {}".format(grid.best_params_))


CPU times: total: 5.91 s
Wall time: 4min 39s
RandomizedSearchCV: Наилучшие значения гиперпараметров {'solver': 'saga', 'penalty': 'l1', 'C': 1}
CPU times: total: 7.14 s
Wall time: 6min 17s
GridSearchCV: Наилучшие значения гиперпараметров {'C': 1, 'penalty': 'l1', 'solver': 'saga'}


Как можно видеть, random search нашел теже параметры, но быстрее

Подберем параметры с помощью optuna и hyperopt

In [5]:
# подбор с помощью optuna
def optuna_rf(trial):
    # задаем пространства поиска гиперпараметров
    penalty = trial.suggest_categorical('penalty', ['l2', 'l1'])
    
    C = trial.suggest_int('C', 1, 10, 1)

    if penalty == 'l2':
        solver = trial.suggest_categorical('solver_l2', ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'])
    else:
       solver = trial.suggest_categorical('solver_l1', [ 'liblinear','saga'])     
    # создаем модель
    model = LogisticRegression(random_state=random_state, n_jobs=-1, max_iter=50, solver=solver, penalty=penalty, C=C)
    # обучаем модель
    model.fit(X, y)

    score = cross_val_score(model, X, y, cv=kf, scoring="f1", n_jobs=-1).mean()
    return score


study = optuna.create_study(
    study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
%time study.optimize(optuna_rf, n_trials=20, show_progress_bar=True)
print("Наилучшие значения гиперпараметров {}".format(study.best_params))

# преобразуем словарь так чтобы избавиться от solver_l1 и solver_l2
if study.best_params.get('solver_l1'):
    best_params= study.best_params
    best_params["solver"] = best_params.pop("solver_l1")
else:
    best_params= study.best_params
    best_params["solver"] = best_params.pop("solver_l2")
model = LogisticRegression(**best_params,random_state=random_state, max_iter=50, n_jobs=-1)
score_optuna = cross_val_score(model, X, y, cv=kf, scoring="f1", n_jobs=-1).mean()

  0%|          | 0/20 [00:00<?, ?it/s]

CPU times: total: 1min 1s
Wall time: 3min 29s
Наилучшие значения гиперпараметров {'penalty': 'l1', 'C': 1, 'solver_l1': 'saga'}


In [6]:
# подбор с помощью hyperopt
#params необходимо для задания комбинаций, тк например sag не рабоает с l1
space = {'C': hp.quniform('C', 1, 10, 1),
         'params': hp.choice('params', [{"penalty": hp.choice('penalty1', ['l1']),
                                         "solver": hp.choice('solver1', ['liblinear', 'saga'])},
                                        {"penalty": hp.choice('penalty2', ['l2']),
                                         "solver": hp.choice('solver2', ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'])}]),
         }


def hyperopt_rf(params, cv=kf, X=X, y=y, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'C': int(params['C']),
              'penalty': params['params']["penalty"],
              'solver': params['params']["solver"]
              }
    model = LogisticRegression(**params, random_state=random_state, max_iter=50)
    model.fit(X, y)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return -score


trials = Trials()

best = fmin(hyperopt_rf,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials,
            rstate=np.random.default_rng(random_state)
            )
print("Наилучшие значения гиперпараметров {}".format(best))

# преобразуем результат функции в словарь
penalty = ['l1','l2']
solver1 = ['liblinear', 'saga']
solver2 = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
if best["params"]==0:
    params = {'C': best["C"], "solver": solver1[best["solver1"]], "penalty": penalty[best["penalty1"]]}
else:
    params = {'C': best["C"], "solver": solver2[best["solver2"]], "penalty": penalty[best["penalty2"]]}
    
model = LogisticRegression(
    **params,
    random_state=random_state, 
    max_iter=50
)
score_hyperopt = cross_val_score(
    model, X, y, cv=kf, scoring="f1", n_jobs=-1).mean()


100%|██████████| 50/50 [08:17<00:00,  9.94s/trial, best loss: -0.7879869198492568]
Наилучшие значения гиперпараметров {'C': 1.0, 'params': 0, 'penalty1': 0, 'solver1': 1}


Теперь сравним метрики

In [7]:
print(f"f1 метрика для GridSearchCV {score_grid}")
print(f"f1 метрика для RandomizedSearchCV {score_random}")
print(f"f1 метрика для optuna {score_optuna}")
print(f"f1 метрика для hyperopt {score_hyperopt}")


f1 метрика для GridSearchCV 0.7879869198492568
f1 метрика для RandomizedSearchCV 0.7879869198492568
f1 метрика для optuna 0.7879869198492568
f1 метрика для hyperopt 0.7879869198492568


Все метрики одинаковы, тк из за небольшой сетки все алгоритмы нашли одни и теже параметры

Теперь обучим случайный лес (для RandomizedSearchCV и GridSearchCV сетка меньше)

In [8]:
paramgrid_log = [{"n_estimators": range(100, 300, 60),
                  "max_depth": range(5, 13, 1),
                  "min_samples_leaf": range(2, 7, 1)}
                 ]

# подбор с помощью RandomizedSearchCV
random = RandomizedSearchCV(
    estimator= RandomForestClassifier(random_state=random_state, n_jobs=-1),
    param_distributions=paramgrid_log,
    cv=kf,
    n_iter=50,
    n_jobs=-1,
    random_state=random_state
)
%time random.fit(X,y)
score_random = cross_val_score(random.best_estimator_, X, y, cv=kf, scoring="f1", n_jobs=-1).mean()
print("RandomizedSearchCV: Наилучшие значения гиперпараметров {}".format(random.best_params_))

# подбор с помощью GridSearchCV
grid = GridSearchCV(
    estimator= RandomForestClassifier(random_state=random_state, n_jobs=-1),
    param_grid=paramgrid_log,
    cv=kf,
    n_jobs=-1
)
%time grid.fit(X,y)
score_grid = cross_val_score(grid.best_estimator_, X, y, cv=kf, scoring="f1", n_jobs=-1).mean()
print("GridSearchCV: Наилучшие значения гиперпараметров {}".format(grid.best_params_))

CPU times: total: 4.8 s
Wall time: 3min 6s
RandomizedSearchCV: Наилучшие значения гиперпараметров {'n_estimators': 220, 'min_samples_leaf': 2, 'max_depth': 12}
CPU times: total: 7.73 s
Wall time: 9min 23s
GridSearchCV: Наилучшие значения гиперпараметров {'max_depth': 12, 'min_samples_leaf': 2, 'n_estimators': 220}


Для hyperopt и optuna сетка в разы больше, тк не хотелось, чтобы grid часами работал 

Теперь подберем параметры для леса с помощью hyperopt

In [9]:

space = {'n_estimators': hp.quniform('n_estimators', 100, 300, 1),
         'max_depth': hp.quniform('max_depth', 5, 26, 1),
         'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1),
         'max_features': ['sqrt', 'log2']
         }


def hyperopt_rf(params, cv=kf, X=X, y=y, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']),
              'max_depth': int(params['max_depth']),
              'min_samples_leaf': int(params['min_samples_leaf'])
              }
    model = RandomForestClassifier(**params, random_state=random_state)
    model.fit(X, y)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return -score


trials = Trials()
best = fmin(hyperopt_rf,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials,
            rstate=np.random.default_rng(random_state)
            )
print("Наилучшие значения гиперпараметров {}".format(best))
model = RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
score_hyperopt = cross_val_score(model, X, y, cv=kf, scoring="f1", n_jobs=-1).mean()

100%|██████████| 50/50 [07:41<00:00,  9.24s/trial, best loss: -0.8220425766820691]
Наилучшие значения гиперпараметров {'max_depth': 25.0, 'min_samples_leaf': 2.0, 'n_estimators': 131.0}


И с помощью optuna

In [10]:
def optuna_rf(trial):
    # задаем пространства поиска гиперпараметров
    n_estimators = trial.suggest_int('n_estimators', 100, 300, 1)
    max_depth = trial.suggest_int('max_depth', 5, 26, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

    # создаем модель
    model = RandomForestClassifier(n_estimators=n_estimators,
                                   max_depth=max_depth,
                                   min_samples_leaf=min_samples_leaf,
                                   random_state=random_state)
    # обучаем модель
    model.fit(X, y)

    score = cross_val_score(model, X, y, cv=kf, scoring="f1", n_jobs=-1).mean()
    return score


study = optuna.create_study(
    study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=50, show_progress_bar=True)
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
model = RandomForestClassifier(**study.best_params,random_state=random_state)
score_optuna = cross_val_score(model, X, y, cv=kf, scoring="f1", n_jobs=-1).mean() 

  0%|          | 0/50 [00:00<?, ?it/s]

Наилучшие значения гиперпараметров {'n_estimators': 237, 'max_depth': 23, 'min_samples_leaf': 2}


Сравним метрики

In [11]:
print(f"f1 метрика для GridSearchCV {score_grid}")
print(f"f1 метрика для RandomizedSearchCV {score_random}")
print(f"f1 метрика для optuna {score_optuna}")
print(f"f1 метрика для hyperopt {score_hyperopt}")


f1 метрика для GridSearchCV 0.8189987020859875
f1 метрика для RandomizedSearchCV 0.8189987020859875
f1 метрика для optuna 0.8240513432365202
f1 метрика для hyperopt 0.8220425766820691


Лично мне больше понравилась работа с optuna, постараюсь ее чаще использовать в дальнейшем (если конечно нужно перебрать небольшое количесво вариантов, будет целесообразнее использовать GridSearchCV)